In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [4]:
def hbr_srch(queries:list, pages:int=1) ->pd.DataFrame:

  df = pd.DataFrame()
  articles_id = set()
  for query in queries:
    for p in range(1,pages+1):
      time.sleep(1)
      url = 'https://habr.com'
      params = {
          'q': query
      }
      headers = {
          'Accept-Language': 'ru,en-us;q=0.7,en;q=0.3'
      }
      with requests.get(url+f'/search/page{p}',params=params, headers=headers) as response:
        soup = BeautifulSoup(response.text)

      articles = soup.find_all('article', 'tm-articles-list__item')
      rows = []

      for elem in articles:
        if elem.get('id') in articles_id:
          continue
        articles_id.add(elem.get('id'))
        data = elem.find('time').get('title')
        title = elem.find('h2').find('span').text
        link = url + elem.find('h2').find('a').get('href')

        with requests.get(link) as res:
          inner_soup = BeautifulSoup(res.text)

        is_expired = inner_soup.find('p', 'tm-expired-company__text')

        if is_expired:
          text_content = is_expired.text
          likes = elem.find('span', 'tm-votes-meter__value').text
        else:
          text_content = inner_soup.find('div','article-formatted-body').text.strip()
          likes = elem.find('span', 'tm-votes-meter__value').text

        rows.append({'data':data, 'title': title, 'link': link, 'text_content':text_content, 'likes':likes})

      inner_df = pd.DataFrame(rows)
      df = pd.concat([df,inner_df], ignore_index=True)

  return df

In [5]:
df = hbr_srch(['deep learning','deep learning', 'sfsfsfsfsfsf', 'python'],3)

In [7]:
pd.set_option('display.max_rows',100)
df.head(100)

,data,title,link,text_content,likes
0,"2021-03-16, 22:49",Круглый стол «Искусственный Интеллект – Machin...,https://habr.com/ru/companies/simbirsoft/news/...,В чем разница между искусственным интеллектом ...,+3
1,"2023-11-18, 18:06",Вкатываемся в Machine Learning с нуля за ноль ...,https://habr.com/ru/articles/774844/,Всем доброго времени суток. Я давно обещала вы...,+155
2,"2017-04-06, 14:02",Быстрый старт: обзор основных Deep Learning фр...,https://habr.com/ru/companies/newprolab/articl...,"Привет, Хабр! Предлагаем вам перевод поста “Ge...",+17
3,"2016-10-05, 18:21",Обзор курсов по Deep Learning,https://habr.com/ru/companies/newprolab/articl...,"Привет, Хабр! Последнее время все больше и бол...",+44
4,"2023-08-09, 12:45",Harnessing the Power of Machine Learning in Fr...,https://habr.com/ru/articles/753498/,Picture this: A thriving e-commerce platform f...,+11
5,"2020-01-07, 14:14",Deep Reinforcement Learning: как научить пауко...,https://habr.com/ru/articles/483078/,"Сегодня я расскажу, как я применил алгоритмы г...",+26
6,"2014-06-16, 13:31","Про котиков, собак, машинное обучение и deep l...",https://habr.com/ru/articles/226347/,«В 1997 году Deep Blue обыграл в шахматы Каспа...,+95
7,"2019-07-12, 15:26",Глубокое обучение (Deep Learning): обзор,https://habr.com/ru/companies/otus/articles/45...,Всем привет. Уже в этом месяце в ОТУС стартует...,+5
8,"2016-09-19, 10:49",DetectNet: Deep Neural Network для Object Dete...,https://habr.com/ru/articles/310332/,Привет Хабр. В последнее время мне очень нрави...,+16
9,"2015-01-28, 14:53",Deep learning и Caffe на новогодних праздниках,https://habr.com/ru/articles/249089/,Мотивация\r\nВ данной статье вы познакомитесь ...,+21
